In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
from keras.datasets import cifar10
import numpy.random as rng

Using TensorFlow backend.


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
from keras.datasets import cifar10
import numpy.random as rng

In [4]:
(x_train,y_train),(x_test,y_test)=cifar10.load_data()

In [5]:
x_train.shape

(50000, 32, 32, 3)

In [6]:
image_list = np.split(x_train[:50000],50000)
label_list = np.split(y_train[:50000],50000)
print(image_list[0].shape)
print(label_list[0].shape)

(1, 32, 32, 3)
(1, 1)


In [7]:
for i in range(len(label_list)):
    label_list[i] = label_list[i][0][0]
    
len(label_list)

50000

In [8]:
# # want to make an array (N,2,h,w,3)
# len(label_list)
_,x,y,z=image_list[2800].shape

In [9]:
for i in range(len(label_list)):
    image_list[i] = image_list[i].reshape((x,y,z))

In [10]:
image_list[0].shape

(32, 32, 3)

In [11]:
X1 = []
X2 = []
label = []
c1=0
c2=0

for i in range(len(label_list)):
    if c1<30:
        idx_1 = rng.randint(0,len(label_list))         
        idx_2 = rng.randint(0,len(label_list))       
        if label_list[idx_1] ==1 and  label_list[idx_2]==1:
            X1.append(image_list[idx_1])
            X2.append(image_list[idx_2])
            label.append(1)
            c1+=1  
            
    if c2<30:
        idx_1 = rng.randint(0,len(label_list))
        idx_2 = rng.randint(0,len(label_list))        
        if label_list[idx_1] != label_list[idx_2]:
            X1.append(image_list[idx_1])
            X2.append(image_list[idx_2])
            label.append(0)
            c2+=1
    

In [12]:
print(X1[0].shape)
print(label[2])
print(len(X1))

(32, 32, 3)
0
60


In [13]:
cnt=0
for i in range(60):
    if label[i]==1:
        cnt+=1
print(cnt)

30


In [14]:
X1= np.array(X1) #convert list to array
X2= np.array(X2)
print(X1.shape)
print(X2.shape)

(60, 32, 32, 3)
(60, 32, 32, 3)


In [15]:
label = np.array(label)

In [16]:
print(label.shape)

(60,)


# MODEL

In [17]:
import os,sys
#os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from keras.engine.topology import Layer
import numpy as np
from keras.layers import initializers,constraints
import tensorflow as tf
from keras.models import Sequential,Model
from keras.utils import conv_utils
from keras import backend as K

from keras.layers import Dense, Dropout, Flatten,Input
from keras.layers import Conv2D, MaxPooling2D,Concatenate,Input
from keras.optimizers import SGD

from keras_contrib.losses import DSSIMObjective
from keras.constraints import *
from keras.layers import Activation, Dense,Dropout
#import pandas
#from utils import *
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from keras.callbacks import ModelCheckpoint
#from morph_layers2D import *
from matplotlib.colors import ListedColormap
import keras


from keras.layers import Conv2D, BatchNormalization, MaxPooling2D, UpSampling2D, Add, Dropout, concatenate
import time
import numpy as np 
import cv2 as cv

In [258]:
SHAPE_Y,SHAPE_X=32,32
inputA = Input(shape=(SHAPE_Y,SHAPE_X,3))
inputB = Input(shape=(SHAPE_Y,SHAPE_X,3))

def comp_model_conv(shape=(SHAPE_Y,SHAPE_X,3)):

    actFunc='relu'    
    ip = Input(shape=(SHAPE_Y,SHAPE_X,3))
    # the first branch operates on the first input
    x=Conv2D(256,(9,9),name='conv11',activation=actFunc, padding='valid',strides=(1,1))(ip)
    x= MaxPooling2D(pool_size=(2, 2), strides=(1,1), padding='valid')(x)#actFunc
    #x = Dropout(0.2)(x)
    
    x=Conv2D(256,(9,9),name='conv12',activation=actFunc, padding='valid',strides=(2,2))(x)#32
    x= MaxPooling2D(pool_size=(2, 2),strides=(1,1), padding='valid')(x)
    x = Dropout(0.2)(x)
    
    x=Conv2D(32,(3,3),name='conv13',activation=actFunc, padding='valid',strides=(2,2))(x)#8
    x= MaxPooling2D(pool_size=(2, 2), strides=(1,1),padding='same')(x)
    
    #x=Conv2D(64,(2,2),name='conv14',activation=actFunc, padding='valid',strides=(1,1))(x)#8
    #x= MaxPooling2D(pool_size=(2, 2), strides=(1,1),padding='same')(x)
    
    x = Flatten()(x)
    
    #x=Dense(160)(x)
    
    #x=Dense(160)(x)
    x=Dense(10, kernel_regularizer=regularizers.l2(0.04))(x)

    #x=Dense(10)(x)
    
    #y= Flatten()(x)
    #print(y.shape)
    model=Model(inputs=[ip],outputs=[x])
    model.summary()
    return model

cmodel=comp_model_conv()
out1=cmodel(inputA)
out2=cmodel(inputB)

L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
combined = L1_layer([out1, out2])
combined = Dense(1, activation = 'sigmoid')(combined)

model = Model(inputs=[inputA, inputB], outputs=[combined])
# print(inputB.shape)
model.compile(loss='mse',optimizer=keras.optimizers.Adam(),metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_57 (InputLayer)        (None, 32, 32, 3)         0         
_________________________________________________________________
conv11 (Conv2D)              (None, 24, 24, 256)       62464     
_________________________________________________________________
max_pooling2d_38 (MaxPooling (None, 23, 23, 256)       0         
_________________________________________________________________
conv12 (Conv2D)              (None, 8, 8, 256)         5308672   
_________________________________________________________________
max_pooling2d_39 (MaxPooling (None, 7, 7, 256)         0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 7, 7, 256)         0         
_________________________________________________________________
conv13 (Conv2D)              (None, 3, 3, 32)          73760     
__________

In [259]:
model.fit([X1/255,X2/255], label, batch_size=20, epochs=60, verbose=1)

Epoch 1/60
60/60 [==============================] - 5s 83ms/step - loss: 1.0409 - acc: 0.5167
Epoch 2/60
60/60 [==============================] - 4s 65ms/step - loss: 1.0236 - acc: 0.5167
Epoch 3/60
60/60 [==============================] - 4s 62ms/step - loss: 0.9346 - acc: 0.5667
Epoch 4/60
60/60 [==============================] - 4s 63ms/step - loss: 0.8824 - acc: 0.6500
Epoch 5/60
60/60 [==============================] - 4s 66ms/step - loss: 0.8385 - acc: 0.7500
Epoch 6/60
60/60 [==============================] - 4s 67ms/step - loss: 0.7806 - acc: 0.8000
Epoch 7/60
60/60 [==============================] - 4s 68ms/step - loss: 0.7211 - acc: 0.8167
Epoch 8/60
60/60 [==============================] - 4s 65ms/step - loss: 0.6523 - acc: 0.9000
Epoch 9/60
60/60 [==============================] - 4s 68ms/step - loss: 0.6374 - acc: 0.8667
Epoch 10/60
60/60 [==============================] - 4s 67ms/step - loss: 0.5856 - acc: 0.8500
Epoch 11/60
60/60 [==============================] - 4s 69m

In [343]:
x_test1 = []
x_test2 = []
c1=0
a = np.zeros((10,),dtype='int64')
testing_image_index = rng.randint(0,len(label_list))         

print("Labels:", end=" ")
for i in range(len(label_list)):
    if c1<10:
        idx_2 = rng.randint(0,len(label_list))
        if a[label_list[idx_2]]==0:
            x_test1.append(image_list[testing_image_index])
            x_test2.append(image_list[idx_2])
            a[label_list[idx_2]]+=1            
            c1+=1
            print(label_list[idx_2],end="  ")
    
print("\nTesting image label:", label_list[testing_image_index])

Labels: 6  4  8  2  5  0  9  3  7  1  
Testing image label: 7


In [344]:
x_test1 = np.array(x_test1)
x_test2 = np.array(x_test2)
t = model.predict([x_test1,x_test2])
t

array([[1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.]], dtype=float32)

In [318]:
t=np.round(t)
t

array([[0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.]], dtype=float32)